##                      Imagen Médica              -                Máster Visión Artificial (2023-2024)

**María Cornejo Antonaya (m.cornejo.2023@alumnos.urjc.es)**

**Nuria Miralles Gavara (n.miralles.2023@alumnos.urjc.es)**

**Juan Montes Cano (juan.montes@urjc.es)**

## Práctica 2: Segmentación de imagen

En la presente práctica, se analizan e implementan diversos algoritmos de segmentación de imagen.


### Ejercicio A) Crecimiento de regiones

El método de crecimiento de regiones se basa en la selección de una semilla en la imagen y la definición de un rango de nivel de gris que caracteriza a la región de interés. A partir de esta semilla y el rango especificado, se procede a expandir iterativamente la región, agregando píxeles adyacentes que cumplen con el criterio de pertenecer al rango de nivel de gris establecido. Este proceso se detiene cuando ya no se pueden agregar más píxeles que cumplan con las condiciones definidas.

En el script el usuario podrá elegir donde se inicializa la semilla, y podrá ver como es incremental la segmentación descomentando las líneas 46 y 47 del script `ejer1.py`.

In [ ]:
import cv2
import cv2
import numpy as np



def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f'Coordenada: ({x}, {y})')


def region_grow(image, seed_coords, threshold_min, threshold_max):
    # Create an empty mask to store the region
    mask = np.zeros_like(image, dtype=np.uint8)
    
    # Create a set to store the coordinates of pixels that have been processed
    processed_pixels = set()
    
    # Get the seed pixel value
    seed_value = image[seed_coords[1], seed_coords[0]]
    
    # Create a queue to store the coordinates of pixels to be processed
    queue = []
    queue.append(seed_coords)
    
    # Process the queue until it's empty
    while queue:
        # Get the next pixel coordinates from the queue
        x, y = queue.pop(0)
        
        # Check if the pixel is within the image boundaries and has not been processed before
        if x >= 0 and x < image.shape[1] and y >= 0 and y < image.shape[0] and (x, y) not in processed_pixels:
            # Mark the pixel as processed
            processed_pixels.add((x, y))
            # Check if the pixel value is within the threshold
            if threshold_min <= image[y, x][0] <= threshold_max:
                
                # Set the pixel value in the mask
                mask[y, x] = 255
                
                # Add the neighboring pixels to the queue
                queue.append((x - 1, y))
                queue.append((x + 1, y))
                queue.append((x, y - 1))
                queue.append((x, y + 1))
        #cv2.imshow("Mask", mask)
        #cv2.waitKey(1)

    return mask

# Path to the image file
image_path = "MaterialP2/hueso.tif"

# Read the image using OpenCV
image = cv2.imread(image_path)

# Check if the image was successfully loaded
if image is not None:
    # Display the image
    cv2.imshow("Image", image)
    # Establecer la función de devolución de llamada para el evento de clic del mouse
    cv2.setMouseCallback('Image', click_event)
    # Wait for a key press
    cv2.waitKey(0)
    
    '''
    seed_x = int(input("Enter the x coordinate of the seed pixel: "))
    seed_y = int(input("Enter the y coordinate of the seed pixel: "))'''

    seed_x = 153
    seed_y = 281
    print(seed_x, seed_y)

    # Get the threshold from the user
    '''
    threshold_max = int(input("Enter the threshold value max: "))
    threshold_min = int(input("Enter the threshold value min: "))
    '''



    threshold_max = 180 # 220 te detecta mas hueso
    threshold_min = 110
    
    # Perform region growing
    mask = region_grow(image, (seed_x, seed_y), threshold_min, threshold_max)
    
    # Display the mask
    cv2.imshow("Mask", mask)
    
    # Wait for a key press
    cv2.waitKey(0)
    
    # Close all windows
    cv2.destroyAllWindows()
else:
    print("Failed to load the image.")



### Resultados

Se muestra como la zona ha ido creciendo para realizar la segmentación:


<img src="A1.jpg" alt="Descripción de la imagen" width="700"/>


<img src="A2.jpg" alt="Descripción de la imagen" width="700"/>


<img src="A3.jpg" alt="Descripción de la imagen" width="700"/>


### Ejercicio B) Segmentación usando algoritmo EM

En este apartado se utiliza el algoritmo EM para la segmentación. Considerando el histograma de una imagen formado por gaussianas de cada clase de la imagen, se lleva a cabo la segmentación buscando las gaussianas que definan mejor el histograma. Este algoritmo maximiza la verosimilitud y permite incluir información a priori.

Para desarollar este apartado se ha utilizado la imagen del corte axial de un cerebro facilitado, brain.bmp. Inicialmente, se ha segmentado la imagen con 4 y 6 clases. Obteniendo un resultado nulo con 4 clases y mejorandolo con 6 clases obteniendo una segmentación más precisa.

TO-DO LOS APARTADITOS


In [ ]:
codigo

### Ejercicio C) Segmentación usando el algoritmo Watershed

El algoritmo Watershed es una técnica de segmentación de imágenes que se utiliza para separar regiones contiguas en una imagen que están separadas por líneas de contorno. La idea principal del algoritmo es imaginar la intensidad de los píxeles de la imagen como una superficie topográfica, donde los mínimos locales corresponden a los valles y las cuencas de agua representan las regiones que deseamos segmentar. Inicialmente, se marcan los mínimos locales como puntos de inicio de inundación, luego se simula un proceso de inundación desde estos puntos, donde se llenan las cuencas de agua y se encuentran los límites naturales entre las regiones.

De esta manera, el algoritmo Watershed divide la imagen en regiones significativas, creando una segmentación que refleja los cambios abruptos en la intensidad de los píxeles. Sin embargo, este método puede producir una sobresegmentación, donde se generan regiones demasiado pequeñas y numerosas, lo que requiere pasos adicionales, como la imposición de mínimos locales, para obtener una segmentación más precisa y útil.

In [ ]:
% Leer la imagen 
imagen = imread('higado.bmp');

% Imagen en escala de grises
imagen_gris = rgb2gray(imagen);

% Paso 1: Calcular la imagen de gradiente
mascara = fspecial('sobel'); % Crear la mascara de Sobel
gradiente_x = imfilter(imagen_gris, mascara'); % Calcular el gradiente en dirección x
gradiente_y = imfilter(imagen_gris, mascara); % Calcular el gradiente en dirección y
gradiente = sqrt(double(gradiente_x).^2 + double(gradiente_y).^2); % Calcular el modulo de los gradientes

% Paso 2: Aplicar Watershed a la imagen de gradiente
transformada_distancia = bwdist(~imbinarize(gradiente)); % Transformada de distancia
transformada_distancia = -transformada_distancia;
transformada_distancia(~imbinarize(gradiente)) = -Inf; % Establecer los minimos locales
segmentacion = watershed(transformada_distancia); % Aplicar Watershed
segmentacion(~mascara) = 0;

% Paso 3: Utilizar imimposemin para imponer mínimos en las zonas adecuadas
% Operaciones de morfologia matematica para limpiar la imagen y mejorar la segmentacion
I = imagen_gris;
se = strel("disk",20);
Io = imopen(I,se);
Ie = imerode(I,se);
Iobr = imreconstruct(Ie,I);
Ioc = imclose(Io,se);
Iobrd = imdilate(Iobr,se);
Iobrcbr = imreconstruct(imcomplement(Iobrd),imcomplement(Iobr));
Iobrcbr = imcomplement(Iobrcbr);

% Identificar maximos regionales
fgm = imregionalmax(Iobrcbr);
se2 = strel(ones(5,5));
fgm2 = imclose(fgm,se2);
fgm3 = imerode(fgm2,se2);
fgm4 = bwareaopen(fgm3,20);

% Binarizar la imagen de fondo
bw = imbinarize(Iobrcbr);
D = bwdist(bw);
DL = watershed(D);
bgm = DL == 0;

% Imponer minimos locales en las regiones adecuadas
minimos = imimposemin(gradiente, bgm | fgm4);

% Paso 4: Obtener el resultado de Watershed sobre la imagen modificada
segmentacion_final = watershed(minimos);

% Visualizacion de los resultados
figure;
subplot(2,2,1); imshow(imagen); title('Imagen Original');
subplot(2,2,2); imshow(gradiente); title('Imagen de Gradiente');
subplot(2,2,3); imshow(label2rgb(segmentacion,'jet',[.5 .5 .5])); title('Resultado Watershed');
subplot(2,2,4); imshow(label2rgb(segmentacion_final)); title('Resultado con mínimos impuestos');


### Resultado

Como se puede observar, al aplicar Watershed directamente sobre la magnitud del gradiente se produce una sobresegamentación. En cambio, al imponer mínimos locales se obtiene un resultado mejorado, diferenciando las diferentes zonas de la imagen. Aun así, el resultado sigue sin ser muy preciso, lo que sugiere que pueden ser necesarios más ajustes o técnicas adicionales para obtener una segmentación más adecuada.

<img src="result_ejer_C.png" alt="Descripción de la imagen" width="700"/>


### Ejercicio D) Segmentación usando el modelo de Chan_Vese

El modelo de segmentación Chan-Vese es un modelo deformable que divide la imagen en dos regiones con máxima diferencia en media de gris. Este modelo evoluciona una superficie en función de las energías medidas, en concreto la diferencia de puntos de dentro y fuera de la región segmentada, junto con una energía medida de la curvatura de la superficie.

Para comenzar, se han analizada y modificado los parámetros del algoritmo proporcionados en los archivos `chanvese.m` y `chanvese_demo.m`; aumentando el valor de la máscara inicial para lograr una mejora en la segmentación de la imagen.

Después, se ha incluido un criterio de parada en la función de los puntos analizados en el presente y el pasado, en comparación con un umbral establecido, con el propósito de determinar si la segmentación se ha estabilizado y si los pasos siguientes pueden generar sobresegmentación

Por último, se ha limitado el tamaño máximo de la región segmentada para controlar dicha región segmentada y evitar que abarque zonas no deseadas.


In [ ]:
code

### Ejercicio E) Segmentación usando atlas

En este apartado, abordaremos la segmentación de imágenes médicas utilizando un enfoque basado en atlas. Nos centraremos en cómo segmentar una imagen utilizando otra imagen ya etiquetada como referencia. La tarea consiste en desarrollar un código que segmente la imagen MR mediante la asignación de etiquetas basadas en la similitud entre parches de las imágenes. En el contexto de imágenes del cerebro, se buscarán parches en una vecindad de 11x11x11 alrededor de cada punto de interés, utilizando un kernel de 3x3x3 para evaluar la similitud.

El algoritmo es muy lento, y para evaluar regiones suficientemente grandes no es viable, por lo que se ha escogido una ROI dentro del volumen tridimensional sobre el aplicamos la segmentación.

El algoritmo toma un punto a clasificar, y en una vecindad de 11x11x11 en el mismo punto en la imagen del atlas, busca la mayor similitud en kernels de 3x3x3. Esta similitud se realiza usando la diferencia media en dichos kernels con respecto al punto original. El punto que tenga la menor diferencia será el que de la etiqueta para clasificar.


In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
# Leer las imágenes
mr_image = sitk.ReadImage("MaterialP2/MR.nii")
ft1_image = sitk.ReadImage("MaterialP2/fT1.nii")
flabels_image = sitk.ReadImage("MaterialP2/fLabels.nii")

# Convertir las imágenes a arrays numpy
mr_array = sitk.GetArrayFromImage(mr_image)
#print(mr_array.shape)
ft1_array = sitk.GetArrayFromImage(ft1_image)
flabels_array = sitk.GetArrayFromImage(flabels_image)

# Crear un subplot con 1 fila y 3 columnas
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Las imágenes volumétricas
images = [mr_array, ft1_array, flabels_array]
titles = ['MR', 'fT1', 'fLabels']
index = 15
# Mostrar las primeras tres imágenes en un solo plot
for i in range(3):
    axes[i].imshow(images[i][index, :, :], cmap='gray')  # Seleccionar la imagen correspondiente al elemento 15
    axes[i].set_title(titles[i])
    axes[i].axis('off')

plt.show()

# Definir las coordenadas de la ROI en mr_array (ejemplo)
x_start_roi, y_start_roi, z_start_roi = 20, 50, 50
x_end_roi, y_end_roi, z_end_roi = 30, 150, 150

# Crear una matriz vacía para almacenar los resultados en la ROI
resultados_roi = np.empty((x_end_roi - x_start_roi, y_end_roi - y_start_roi, z_end_roi - z_start_roi))

# Iterar sobre cada voxel dentro de la ROI
for x in range(x_start_roi, x_end_roi):
    for y in range(y_start_roi, y_end_roi):
        for z in range(z_start_roi, z_end_roi):
            # Coordenadas del voxel de referencia en la imagen derecha
            x_ref, y_ref, z_ref = x, y, z
            
            # Definir las coordenadas de la región 11x11x11 en la imagen derecha
            x_start = max(x_ref - 5, x_start_roi)
            x_end = min(x_ref + 6, x_end_roi)
            y_start = max(y_ref - 5, y_start_roi)
            y_end = min(y_ref + 6, y_end_roi)
            z_start = max(z_ref - 5, z_start_roi)
            z_end = min(z_ref + 6, z_end_roi)

            # Inicializar el voxel más similar y su diferencia mínima
            voxel_mas_similar = None
            diferencia_minima = float('inf')

            # Iterar sobre la región 11x11x11 en la imagen derecha
            for xi in range(x_start, x_end):
                for yi in range(y_start, y_end):
                    for zi in range(z_start, z_end):
                        # Definir las coordenadas del kernel 3x3x3 alrededor del voxel de referencia
                        x_kernel_start = max(xi - 1, x_start)
                        x_kernel_end = min(xi + 2, x_end)
                        y_kernel_start = max(yi - 1, y_start)
                        y_kernel_end = min(yi + 2, y_end)
                        z_kernel_start = max(zi - 1, z_start)
                        z_kernel_end = min(zi + 2, z_end)

                        # Calcular la diferencia media en el kernel 3x3x3
                        diferencia_media = np.mean(np.abs(mr_array[x:x+1, y:y+1, z:z+1] - mr_array[x_kernel_start:x_kernel_end, y_kernel_start:y_kernel_end, z_kernel_start:z_kernel_end]))
                        
                        # Actualizar el voxel más similar si encontramos una diferencia mínima
                        if diferencia_media < diferencia_minima:
                            diferencia_minima = diferencia_media
                            voxel_mas_similar = flabels_array[xi, yi, zi]  # Voxel más similar encontrado

            # Almacenar el voxel más similar en los resultados
            resultados_roi[x - x_start_roi, y - y_start_roi, z - z_start_roi] = voxel_mas_similar



# Elementos a mostrar
elementos_a_mostrar = [0, 5, 9]

# Mostrar los elementos seleccionados
for idx, elemento in enumerate(elementos_a_mostrar, 1):
    plt.subplot(1, len(elementos_a_mostrar), idx)
    plt.imshow(resultados_roi[elemento, :, :], cmap='gray')  # Seleccionar el plano específico
    plt.colorbar()
    plt.title(f"Elemento {elemento}")
    plt.axis('off')

plt.show()

### Resultado

Imagen en la ROI `[20:30],[50,150],[50,150]`, se muestran las slices 0,5,9 que se corresponden con las slices 20,25,29 de la imagen original

<img src="ejerE.png" alt="Descripción de la imagen" width="700"/>
